## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [12]:
# If you didn't download it yet, please get the relevant file now!
!wget "https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt" -P "./05data/"

--2020-11-10 15:23:39--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.1, 2620:100:6025:1::a27d:4501
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2020-11-10 15:23:40--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfdd226d66efde2398815e7430a.dl.dropboxusercontent.com/cd/0/inline/BC43DJ1kmarGJKr8G_joqhGtTl8mTv71xSkHmCy0KeHfATExezqUWvtQ9fEwUtiYYF-1AyYoTc8vH6iXinkWMK4ckO05ttcRpZUDCuppap9-ClvYGQ5vmr_aoTTVXNKvZXA/file# [following]
--2020-11-10 15:23:40--  https://ucfdd226d66efde2398815e7430a.dl.dropboxusercontent.com/cd/0/inline/BC43DJ1kmarGJKr8G_joqhGtTl8mTv71xSkHmCy0KeHfATExezqUWvtQ9fEwUtiYYF-1AyYoTc8vH6iXinkWMK4ckO05ttcRpZUDCu

### 1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

In [2]:
import numpy as np
import pandas as pd

In [3]:
N = 20000
data = pd.read_csv("./05data/data_000637.txt",nrows=N)
#data["ORBIT_CNT"][:50].plot()

In [4]:
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
19995,1,0,27,3869200366,2513,29
19996,1,0,63,3869200366,2517,6
19997,1,0,32,3869200366,2519,5
19998,1,0,17,3869200366,2522,21


### 2\. Find out the number of BX in a ORBIT (the value 'x').

If I correctly understood that the BX_COUNTER  is resetted after the increase of ORBIT_CNT, then the BX count associated to a given orbit can be obtained as follows:

In [13]:
import matplotlib.pyplot as plt

vs  = np.array(data.groupby("BX_COUNTER").max()["TDC_MEAS"])
vs.mean(),vs.std()

(23.468054763262977, 6.036892712829014)

In [16]:
BX_of_ORBIT = data.groupby("ORBIT_CNT").max()["BX_COUNTER"]
print(BX_of_ORBIT)

ORBIT_CNT
3869200167    3187
3869200168    3538
3869200169    2766
3869200170    3377
3869200171    3542
              ... 
3869200362    3425
3869200363    3460
3869200364    3352
3869200365    2787
3869200366    2522
Name: BX_COUNTER, Length: 200, dtype: int64


### 3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

For each BX_COUNTER value I take the max of the corresponding TDC_MEAS

In [12]:
data.groupby("ORBIT_CNT").sum()

,HEAD,FPGA,TDC_CHANNEL,BX_COUNTER,TDC_MEAS
ORBIT_CNT,,,,,
3869200167,1,1,139,3187,28
3869200168,1,1,139,3538,29
3869200169,1,1,139,2766,29
3869200170,1,1,139,3377,29
3869200171,1,1,139,3542,29
...,...,...,...,...,...
3869200362,1,1,139,3425,29
3869200363,1,1,139,3460,29
3869200364,1,1,139,3352,29


I sum the maximum BX_count reached for each orbit, and multiply for the time value of each BX

In [70]:
print("measuring time for the first {} data: ".format(N), BX_of_ORBIT.sum()*25/10e9, "s")

measuring time for the first 20000 data:  0.0016212175 s


### 4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

In [ ]:
data.append



5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139